In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

c:\Users\Joaquín Amat\Documents\GitHub\skforecast


In [2]:
# Data processing
# ==============================================================================
import numpy as np
import pandas as pd

# Plots
# ==============================================================================
import matplotlib.pyplot as plt
from skforecast.plot import set_dark_theme
from skforecast.plot import plot_prediction_intervals
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.stattools import pacf
from skforecast.plot import plot_residuals
import plotly.graph_objects as go
import plotly.io as pio
import plotly.offline as poff
pio.templates.default = "seaborn"
pio.renderers.default = 'notebook' 
poff.init_notebook_mode(connected=True)

# Modelling and Forecasting
# ==============================================================================
import skforecast
import sklearn
from sklearn.linear_model import Ridge
from lightgbm import LGBMRegressor
from sklearn.feature_selection import RFECV
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_pinball_loss
from feature_engine.datetime import DatetimeFeatures
from feature_engine.creation import CyclicalFeatures
from feature_engine.timeseries.forecasting import LagFeatures
from feature_engine.timeseries.forecasting import WindowFeatures
from skforecast.direct import ForecasterDirect
from skforecast.model_selection import TimeSeriesFold, OneStepAheadFold
from skforecast.model_selection import bayesian_search_forecaster
from skforecast.model_selection import backtesting_forecaster
from skforecast.feature_selection import select_features
from skforecast.preprocessing import RollingFeatures
from skforecast.metrics import coverage

# Warnings configuration
# ==============================================================================
import warnings
warnings.filterwarnings('once')

print('Versión skforecast:', skforecast.__version__)
print('Versión sklearn:', sklearn.__version__)

Versión skforecast: 0.15.0
Versión sklearn: 1.5.2


In [3]:
# Load data
# ==============================================================================
data = pd.read_csv("https://raw.githubusercontent.com/skforecast/skforecast-datasets/main/data/ETTm2.csv")
data['date'] = pd.to_datetime(data['date'])
data = data.set_index('date')
data = data.asfreq('15min')
data = data.resample(rule="1h", closed="left", label="right").mean()
data

,HUFL,HULL,MUFL,MULL,LUFL,LULL,OT
date,,,,,,,
2016-07-01 01:00:00,38.784501,10.88975,34.753500,8.55100,4.12575,1.26050,37.838250
2016-07-01 02:00:00,36.041249,9.44475,32.696001,7.13700,3.59025,0.62900,36.849250
2016-07-01 03:00:00,38.240000,11.41350,35.343501,9.10725,3.06000,0.31175,35.915750
2016-07-01 04:00:00,37.800250,11.45525,34.881000,9.28850,3.04400,0.60750,32.839375
2016-07-01 05:00:00,36.501750,10.49200,33.708250,8.65150,2.64400,0.00000,31.466125
...,...,...,...,...,...,...,...
2018-06-26 16:00:00,39.517250,11.45500,50.616000,12.08950,-10.64275,-1.28475,47.744249
2018-06-26 17:00:00,39.140499,11.32975,49.865250,11.84825,-10.33100,-1.35400,48.183498
2018-06-26 18:00:00,42.428501,12.85850,53.591250,13.33575,-10.95475,-1.41800,47.853999


In [4]:
# Calendar features
# ==============================================================================
features_to_extract = [
    'year',
    'month',
    'week',
    'day_of_week',
    'hour'
]
calendar_transformer = DatetimeFeatures(
    variables           = 'index',
    features_to_extract = features_to_extract,
    drop_original       = False,
)

# Lags of exogenous variables
# ==============================================================================
lag_transformer = LagFeatures(
    variables = ["HUFL", "MUFL", "MULL", "HULL", "LUFL", "LULL"],
    periods   = [1, 2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 24, 42],
)

# Rolling features for exogenous variables
# ==============================================================================
wf_transformer = WindowFeatures(
    variables   = ["HUFL", "MUFL", "MULL", "HULL", "LUFL", "LULL"],
    window      = ["1D", "7D"],
    functions   = ["mean", "max", "min"],
    freq        = "1h",
)

# Cliclical encoding of calendar features
# ==============================================================================
features_to_encode = [
    "month",
    "week",
    "day_of_week",
    "hour",
]
max_values = {
    "month": 12,
    "week": 52,
    "day_of_week": 7,
    "hour": 24,
}
cyclical_encoder = CyclicalFeatures(
                        variables     = features_to_encode,
                        max_values    = max_values,
                        drop_original = True
                   )

exog_transformer = make_pipeline(
                        calendar_transformer,
                        lag_transformer,
                        wf_transformer,
                        cyclical_encoder
                   )
display(exog_transformer)

data = exog_transformer.fit_transform(data)
# Remove rows with NaNs created by lag features
data = data.dropna()
exog_features = data.columns.difference(['OT']).tolist()
display(data.head(3))

Pipeline(steps=[('datetimefeatures',
                 DatetimeFeatures(drop_original=False,
                                  features_to_extract=['year', 'month', 'week',
                                                       'day_of_week', 'hour'],
                                  variables='index')),
                ('lagfeatures',
                 LagFeatures(periods=[1, 2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 14,
                                      15, 16, 17, 18, 19, 20, 21, 23, 24, 42],
                             variables=['HUFL', 'MUFL', 'MULL', 'HULL', 'LUFL',
                                        'LULL'])),
                ('windowfeatures',
                 WindowFeatures(freq='1h', functions=['mean', 'max', 'min'],
                                variables=['HUFL', 'MUFL', 'MULL', 'HULL',
                                           'LUFL', 'LULL'],
                                window=['1D', '7D'])),
                ('cyclicalfeatures',
                 CyclicalFeatures(drop_original=True,
                                  max_values={'day_of_week': 7, 'hour': 24,
                                              'month': 12, 'week': 52},
                                  variables=['month', 'week', 'day_of_week',
                                             'hour']))])

,HUFL,HULL,MUFL,MULL,LUFL,LULL,OT,year,HUFL_lag_1,MUFL_lag_1,...,LULL_window_7D_max,LULL_window_7D_min,month_sin,month_cos,week_sin,week_cos,day_of_week_sin,day_of_week_cos,hour_sin,hour_cos
date,,,,,,,,,,,,,,,,,,,,,
2016-07-02 19:00:00,33.6330,9.08900,29.874751,6.95600,3.753,0.61025,28.719500,2016,32.5440,29.017001,...,1.2605,0.0,-0.5,-0.866025,1.224647e-16,-1.0,-0.974928,-0.222521,-0.965926,0.258819
2016-07-02 20:00:00,31.7065,7.72750,27.884500,5.63600,3.753,0.67425,29.103875,2016,33.6330,29.874751,...,1.2605,0.0,-0.5,-0.866025,1.224647e-16,-1.0,-0.974928,-0.222521,-0.866025,0.500000
2016-07-02 21:00:00,31.8110,7.81125,27.362000,5.18025,4.435,1.42075,29.598500,2016,31.7065,27.884500,...,1.2605,0.0,-0.5,-0.866025,1.224647e-16,-1.0,-0.974928,-0.222521,-0.707107,0.707107


In [5]:
selected_exog = [
    "HUFL",
    "HUFL_lag_1",
    "HUFL_lag_12",
    "HUFL_lag_13",
    "HUFL_lag_15",
    "HUFL_lag_19",
    "HUFL_lag_2",
    "HUFL_lag_20",
    "HUFL_lag_23",
    "HUFL_lag_4",
    "HUFL_lag_5",
    "HUFL_lag_9",
    "HUFL_window_1D_mean",
    "HULL",
    "HULL_lag_1",
    "HULL_lag_12",
    "HULL_lag_14",
    "HULL_lag_15",
    "HULL_lag_2",
    "HULL_lag_20",
    "HULL_lag_21",
    "HULL_lag_23",
    "HULL_lag_3",
    "HULL_lag_4",
    "HULL_window_1D_mean",
    "LUFL",
    "LUFL_lag_1",
    "LUFL_lag_10",
    "LUFL_lag_15",
    "LUFL_lag_19",
    "LUFL_lag_2",
    "LUFL_lag_20",
    "LUFL_lag_23",
    "LUFL_lag_3",
    "LUFL_lag_4",
    "LUFL_lag_5",
    "LUFL_window_1D_mean",
    "LULL",
    "LULL_lag_1",
    "LULL_lag_12",
    "LULL_lag_13",
    "LULL_lag_14",
    "LULL_lag_18",
    "LULL_lag_19",
    "LULL_lag_20",
    "LULL_lag_21",
    "LULL_lag_23",
    "LULL_lag_24",
    "LULL_lag_4",
    "LULL_lag_5",
    "LULL_lag_6",
    "LULL_window_1D_max",
    "LULL_window_1D_min",
    "MUFL",
    "MUFL_lag_1",
    "MUFL_lag_11",
    "MUFL_lag_12",
    "MUFL_lag_13",
    "MUFL_lag_15",
    "MUFL_lag_2",
    "MUFL_lag_20",
    "MUFL_lag_23",
    "MUFL_lag_4",
    "MUFL_lag_9",
    "MUFL_window_1D_mean",
    "MULL",
    "MULL_lag_1",
    "MULL_lag_11",
    "MULL_lag_12",
    "MULL_lag_13",
    "MULL_lag_14",
    "MULL_lag_15",
    "MULL_lag_17",
    "MULL_lag_19",
    "MULL_lag_2",
    "MULL_lag_20",
    "MULL_lag_3",
    "MULL_lag_4",
    "MULL_lag_5",
    "MULL_lag_9",
    "MULL_window_1D_mean",
    "MULL_window_1D_min",
    "MULL_window_7D_mean",
    "day_of_week_sin",
    "hour_cos",
    "hour_sin",
    "month_cos",
    "month_sin",
    "week_cos",
    "week_sin",
    "year",
]

lags = [1, 2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 24, 42]

In [6]:
# Split train-validation-test
# ==============================================================================
end_train = '2017-10-01 23:59:00'
end_validation = '2018-04-03 23:59:00'
data_train = data.loc[: end_train, :]
data_val   = data.loc[end_train:end_validation, :]
data_test  = data.loc[end_validation:, :]

print(f"Dates train      : {data_train.index.min()} --- {data_train.index.max()}  (n={len(data_train)})")
print(f"Dates validacion : {data_val.index.min()} --- {data_val.index.max()}  (n={len(data_val)})")
print(f"Dates test       : {data_test.index.min()} --- {data_test.index.max()}  (n={len(data_test)})")

Dates train      : 2016-07-02 19:00:00 --- 2017-10-01 23:00:00  (n=10949)
Dates validacion : 2017-10-02 00:00:00 --- 2018-04-03 23:00:00  (n=4416)
Dates test       : 2018-04-04 00:00:00 --- 2018-06-26 20:00:00  (n=2013)


In [7]:
# Create forecasters: one for each limit of the interval
# ==============================================================================
# The forecasters obtained for alpha=0.1 and alpha=0.9 produce a 80% confidence
# interval (90% - 10% = 80%).

# Forecaster for quantile 10%
forecaster_lower_bound = ForecasterDirect(
                            regressor = LGBMRegressor(
                                            objective    = 'quantile',
                                            metric       = 'quantile',
                                            alpha        = 0.1,
                                            random_state = 15926,
                                            verbose      = -1
                                            
                                        ),
                            lags  = lags,
                            steps = 24,
                            differentiation = 1,
                        )
# Forecaster for quantile 90%
forecaster_upper_bound = ForecasterDirect(
                            regressor = LGBMRegressor(
                                            objective    = 'quantile',
                                            metric       = 'quantile',
                                            alpha        = 0.9,
                                            random_state = 15926,
                                            verbose      = -1
                                            
                                        ),
                            lags  = lags,
                            steps = 24,
                            differentiation = 1,
                        )

In [8]:
# Loss function for each quantile (pinball_loss)
# ==============================================================================
def mean_pinball_loss_constructor(alpha: float) -> callable:
    """
    Create Pinball loss for a given quantile.

    Parameters
    ----------
    alpha: float
        Quantile.

    Returns
    -------
    mean_pinball_loss_q: callable
        Mean Pinball loss for the given quantile.
    """
    if not (0 <= alpha <= 1):
        raise ValueError("alpha must be between 0 and 1.")

    def mean_pinball_loss_q(y_true, y_pred):
        return mean_pinball_loss(y_true, y_pred, alpha=alpha)
    return mean_pinball_loss_q

mean_pinball_loss_q05 = mean_pinball_loss_constructor(alpha=0.05)
mean_pinball_loss_q10 = mean_pinball_loss_constructor(alpha=0.1)
mean_pinball_loss_q90 = mean_pinball_loss_constructor(alpha=0.9)
mean_pinball_loss_q95 = mean_pinball_loss_constructor(alpha=0.95)

In [9]:
# Bayesian search of hyper-parameters and lags for each quantile forecaster
# ==============================================================================
def search_space(trial):
    search_space  = {
        'n_estimators'  : trial.suggest_int('n_estimators', 100, 500, step=50),
        'max_depth'     : trial.suggest_categorical('max_depth', [-1, 3, 5, 7, 10]),
        'learning_rate' : trial.suggest_float('learning_rate', 0.01, 0.1)
    }

    return search_space

cv = OneStepAheadFold(
        initial_train_size = len(data.loc[:end_train, :]),
        differentiation    = 1,
     )

results_grid_lower_bound = bayesian_search_forecaster(
                       forecaster     = forecaster_lower_bound,
                       y              = data.loc[:end_validation, 'OT'],
                       exog           = data.loc[:end_validation, selected_exog],
                       cv             = cv,
                       metric         = mean_pinball_loss_q10,
                       search_space   = search_space,
                       n_trials       = 10,
                       random_state   = 123,
                       return_best    = True,
                       n_jobs         = 'auto',
                       verbose        = False,
                       show_progress  = True
                   )

results_grid_upper_bound = bayesian_search_forecaster(
                       forecaster    = forecaster_upper_bound,
                       y              = data.loc[:end_validation, 'OT'],
                       exog           = data.loc[:end_validation, selected_exog],
                       cv            = cv,
                       metric        = mean_pinball_loss_q90,
                       search_space  = search_space,
                       n_trials      = 10,
                       random_state  = 123,
                       return_best   = True,
                       n_jobs        = 'auto',
                       verbose       = False,
                       show_progress = True
                   )

c:\Users\Joaquín Amat\Documents\GitHub\skforecast\skforecast\model_selection\_search.py:729: OneStepAheadValidationWarning:

One-step-ahead predictions are used for faster model comparison, but they may not fully represent multi-step prediction performance. It is recommended to backtest the final model for a more accurate multi-step performance estimate. 
 You can suppress this warning using: warnings.simplefilter('ignore', category=OneStepAheadValidationWarning)



  0%|          | 0/10 [00:00<?, ?it/s]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  9 10 11 12 13 14 15 16 17 18 19 20 21 23 24 42] 
  Parameters: {'n_estimators': 400, 'max_depth': 7, 'learning_rate': 0.0982687778546154}
  One-step-ahead metric: 89.37987877915181


c:\Users\Joaquín Amat\Documents\GitHub\skforecast\skforecast\model_selection\_search.py:729: OneStepAheadValidationWarning:

One-step-ahead predictions are used for faster model comparison, but they may not fully represent multi-step prediction performance. It is recommended to backtest the final model for a more accurate multi-step performance estimate. 
 You can suppress this warning using: warnings.simplefilter('ignore', category=OneStepAheadValidationWarning)



  0%|          | 0/10 [00:00<?, ?it/s]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  9 10 11 12 13 14 15 16 17 18 19 20 21 23 24 42] 
  Parameters: {'n_estimators': 400, 'max_depth': 7, 'learning_rate': 0.0982687778546154}
  One-step-ahead metric: 115.96181907747552


In [10]:
# Backtesting on test data
# ==============================================================================
cv = TimeSeriesFold(
        initial_train_size = len(data.loc[:end_validation, :]),
        steps              = 24,  # all hours of next day
        differentiation    = 1
     )
_, predictions_lower_bound = backtesting_forecaster(
                                 forecaster          = forecaster_lower_bound,
                                 y                   = data['OT'],
                                 exog                = data[selected_exog],
                                 cv                  = cv,
                                 metric              = mean_pinball_loss_q10,
                                 n_jobs              = 'auto',
                                 verbose             = False,
                                 show_progress       = True
                              )

_, predictions_upper_bound = backtesting_forecaster(
                                  forecaster          = forecaster_upper_bound,
                                  y                   = data['OT'],
                                  exog                = data[selected_exog],
                                  cv                  = cv,
                                  metric              = mean_pinball_loss_q90,
                                  n_jobs              = 'auto',
                                  verbose             = False,
                                  show_progress       = True
                              )

prediction_interval = pd.concat([predictions_lower_bound, predictions_upper_bound], axis=1)
prediction_interval.columns = ['lower_bound', 'upper_bound']
prediction_interval.head(3)

  0%|          | 0/84 [00:00<?, ?it/s]

  0%|          | 0/84 [00:00<?, ?it/s]

,lower_bound,upper_bound
2018-04-04 00:00:00,32.482983,32.871945
2018-04-04 01:00:00,31.686623,32.403487
2018-04-04 02:00:00,30.733921,32.081329


In [30]:
# Predicted interval coverage (on test data)
# ==============================================================================
empirical_coverage = coverage(
                        y_true      = data.loc[end_validation:, 'OT'].to_numpy(),
                        lower_bound = predictions_lower_bound["pred"].to_numpy(), 
                        upper_bound = predictions_upper_bound["pred"].to_numpy()
                    )
print(f"Predicted interval coverage: {round(100 * empirical_coverage, 2)} %")

# Area of the interval
# ==============================================================================
area = (prediction_interval["upper_bound"] - prediction_interval["lower_bound"]).sum()
print(f"Area of the interval: {round(area, 2)}")

Predicted interval coverage: 88.38 %
Area of the interval: 28378.45


In [12]:
# Plot
# ==============================================================================
fig = go.Figure([
    go.Scatter(name='Real value', x=data_test.index, y=data_test['OT'], mode='lines'),
    go.Scatter(
        name='Upper Bound', x=predictions_upper_bound.index, y=predictions_upper_bound['pred'],
        mode='lines', marker=dict(color="#444"), line=dict(width=0), showlegend=False
    ),
    go.Scatter(
        name='Lower Bound', x=predictions_lower_bound.index, y=predictions_lower_bound['pred'],
        marker=dict(color="#444"), line=dict(width=0), mode='lines',
        fillcolor='rgba(68, 68, 68, 0.3)', fill='tonexty', showlegend=False
    )
])
fig.update_layout(
    title="Real value vs predicted in test data",
    xaxis_title="Date time",
    yaxis_title="OT",
    width=800,
    height=400,
    margin=dict(l=20, r=20, t=35, b=20),
    hovermode="x",
    legend=dict(orientation="h", yanchor="top", y=1.1, xanchor="left", x=0.001)
)
fig.show()

# Conformal intervals

In [15]:
# 1) Backtesting on your calibration set
# ==============================================================================
cv = TimeSeriesFold(
        initial_train_size = len(data.loc[:end_train, :]),
        steps              = 24,  # all hours of next day
        differentiation    = 1
     )
_, predictions_lower_bound = backtesting_forecaster(
                                  forecaster          = forecaster_lower_bound,
                                  y                   = data.loc[:end_validation, 'OT'],
                                  exog                = data.loc[:end_validation, selected_exog],
                                  cv                  = cv,
                                  metric              = mean_pinball_loss_q05,
                                  n_jobs              = 'auto',
                                  verbose             = False,
                                  show_progress       = True
                              )

_, predictions_upper_bound = backtesting_forecaster(
                                  forecaster          = forecaster_upper_bound,
                                  y                   = data.loc[:end_validation, 'OT'],
                                  exog                = data.loc[:end_validation, selected_exog],
                                  cv                  = cv,
                                  metric              = mean_pinball_loss_q05,
                                  n_jobs              = 'auto',
                                  verbose             = False,
                                  show_progress       = True
                              )
prediction_interval_calibration = pd.concat([predictions_lower_bound, predictions_upper_bound], axis=1)
prediction_interval_calibration.columns = ['lower_bound', 'upper_bound']
prediction_interval_calibration['y_true'] = data.loc[end_train:end_validation, 'OT']
prediction_interval_calibration.head(3)


,lower_bound,upper_bound,y_true
2017-10-02 00:00:00,32.259985,32.945106,32.839251
2017-10-02 01:00:00,31.326874,32.804371,32.949501
2017-10-02 02:00:00,30.574089,32.609914,32.729626


In [19]:
# 2) Non-conformity score
# ==============================================================================
# For one interval, the residuals, the difference between the prediction and the
# closest side of the predicted interval, are your non-conformity scores.
# Create a sorted list of the non-conformity scores.
y_true = data.loc[end_train:end_validation, 'OT']
# conformity_scores = np.max(
#     [
#         prediction_interval_calibration['lower_bound'] - y_true,
#         y_true - prediction_interval_calibration['upper_bound'],
#     ],
#     axis=0,
# )
# conformity_scores
diff_upper = prediction_interval_calibration["y_true"] - prediction_interval_calibration["upper_bound"]
diff_lower = prediction_interval_calibration["y_true"] - prediction_interval_calibration["lower_bound"]
conformity_scores = np.where(abs(diff_upper) < abs(diff_lower), diff_upper, diff_lower)
conformity_scores

array([ -0.10585536,   0.14512976,   0.11971177, ..., -12.28048126,
       -13.76437701, -14.6890635 ])

In [21]:
# 3) Correction factor
# ==============================================================================
emperical_quantile = 0.8
correction_factor = np.quantile(conformity_scores, emperical_quantile, method="higher")
correction_factor

-0.6516577613351586

In [ ]:
# 4) Backtesting on test data
# ==============================================================================
cv = TimeSeriesFold(
        initial_train_size = len(data.loc[:end_validation, :]),
        steps              = 24,  # all hours of next day
        differentiation    = 1
     )
_, predictions_lower_bound = backtesting_forecaster(
                                 forecaster          = forecaster_lower_bound,
                                 y                   = data['OT'],
                                 exog                = data[selected_exog],
                                 cv                  = cv,
                                 metric              = mean_pinball_loss_q10,
                                 n_jobs              = 'auto',
                                 verbose             = False,
                                 show_progress       = True
                              )

_, predictions_upper_bound = backtesting_forecaster(
                                  forecaster          = forecaster_upper_bound,
                                  y                   = data['OT'],
                                  exog                = data[selected_exog],
                                  cv                  = cv,
                                  metric              = mean_pinball_loss_q90,
                                  n_jobs              = 'auto',
                                  verbose             = False,
                                  show_progress       = True
                              )

prediction_interval_test = pd.concat([predictions_lower_bound, predictions_upper_bound], axis=1)
prediction_interval_test.columns = ['lower_bound', 'upper_bound']
prediction_interval_test['y_true'] = data.loc[end_validation:, 'OT']
prediction_interval_test.head(3)

,lower_bound,upper_bound,y_true
2018-04-04 00:00:00,32.482983,32.871945,32.674500
2018-04-04 01:00:00,31.686623,32.403487,31.575750
2018-04-04 02:00:00,30.733921,32.081329,29.763125


In [27]:
# 5) Conformal interval
# ==============================================================================
prediction_interval_test['lower_bound_conformal'] = prediction_interval_test['lower_bound'] - correction_factor
prediction_interval_test['upper_bound_conformal'] = prediction_interval_test['upper_bound'] + correction_factor
prediction_interval_test.head(3)

,lower_bound,upper_bound,y_true,lower_bound_conformal,upper_bound_conformal
2018-04-04 00:00:00,32.482983,32.871945,32.674500,33.134641,32.220287
2018-04-04 01:00:00,31.686623,32.403487,31.575750,32.338281,31.751829
2018-04-04 02:00:00,30.733921,32.081329,29.763125,31.385578,31.429671


In [29]:
empirical_coverage = coverage(
                        y_true      = prediction_interval_test['y_true'].to_numpy(),
                        lower_bound = prediction_interval_test["lower_bound_conformal"].to_numpy(), 
                        upper_bound = prediction_interval_test["upper_bound_conformal"].to_numpy()
                    )
print(f"Predicted interval coverage: {round(100 * empirical_coverage, 2)} %")

# Area of the interval
# ==============================================================================
area = (prediction_interval_test["upper_bound_conformal"] - prediction_interval_test["lower_bound_conformal"]).sum()
print(f"Area of the interval: {round(area, 2)}")

Predicted interval coverage: 71.44 %
Area of the interval: 25754.88
